### Efficient Net

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet

In [2]:
batch_size = 32
learning_rate = 0.0001
num_epochs = 15

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std = [0.229,0.224,0.225])                           
                                ])

In [3]:
dataset = datasets.ImageFolder(root='data/Training', transform=transform)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 2) 

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/15], Loss: 0.1410
Epoch [2/15], Loss: 0.0746
Epoch [3/15], Loss: 0.0481
Epoch [4/15], Loss: 0.0299
Epoch [5/15], Loss: 0.0189
Epoch [6/15], Loss: 0.0151
Epoch [7/15], Loss: 0.0130
Epoch [8/15], Loss: 0.0108
Epoch [9/15], Loss: 0.0109
Epoch [10/15], Loss: 0.0080
Epoch [11/15], Loss: 0.0092
Epoch [12/15], Loss: 0.0080
Epoch [13/15], Loss: 0.0070
Epoch [14/15], Loss: 0.0068
Epoch [15/15], Loss: 0.0066


In [ ]:
torch.save(model.state_dict(), 'efficientnet_gender_classification_model.pth')

In [ ]:
import cv2
from PIL import Image
import numpy as np
from facenet_pytorch import MTCNN
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
import gradio as gr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 2)  # Adjust the final layer for 2 classes (male, female)
model.load_state_dict(torch.load('efficientnet_gender_classification_model.pth', map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

mtcnn = MTCNN(keep_all=True, device=device)

def classify_gender(image):
    frame = np.array(image)  
    boxes, _ = mtcnn.detect(frame)

    if boxes is not None:
        for box in boxes:
            # Extract the face region
            x1, y1, x2, y2 = map(int, box)
            face = frame[y1:y2, x1:x2]

            face_pil = Image.fromarray(face)
            face_tensor = transform(face_pil).unsqueeze(0).to(device)

            with torch.no_grad():
                outputs = model(face_tensor)
                _, predicted = torch.max(outputs, 1)
                gender = 'Female' if predicted.item() == 0 else 'Male'

            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, gender, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return frame

def webcam_feed():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = classify_gender(frame)

        yield frame

    cap.release()

def gradio_webcam_feed():
    for frame in webcam_feed():
        yield frame

iface = gr.Interface(fn=gradio_webcam_feed, 
                     inputs=[], 
                     outputs=gr.Image(label="Gender Classification"),
                     live=True)

iface.launch()
